In [11]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent.parent))

In [12]:
import time
import triton
import triton.language as tl
import torch
import torch.nn.functional as F
import pandas as pd
from conv_gemm.triton_kernels.int8.img2col_int8_kernel import img2col_int8_kernel

In [13]:
def img2col_int8(
    x_q,
    Kh, Kw, Sh, Sw, Ph, Pw, Dh, Dw,
    K_pad,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
):
    N, Cin, H, W = x_q.shape

    Ho = (H + 2*Ph - Dh*(Kh - 1) - 1)//Sh + 1
    Wo = (W + 2*Pw - Dw*(Kw - 1) - 1)//Sw + 1
    M = N * Ho * Wo
    K_real = Cin * Kh * Kw

    cols_q = torch.empty((M, K_pad), dtype=torch.int8, device=x_q.device)

    sN, sC, sH, sW = x_q.stride()

    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(K_pad, BLOCK_K))

    img2col_int8_kernel[grid](
        x_q, cols_q,
        N, Cin, H, W,
        Kh, Kw,
        Sh, Sw,
        Ph, Pw,
        Dh, Dw,
        Ho, Wo,
        sN, sC, sH, sW,
        K_real, K_pad,
        BLOCK_M=BLOCK_M,
        BLOCK_K=BLOCK_K,
        num_warps=num_warps,
        num_stages=num_stages
    )

    return cols_q, (Ho, Wo)

In [14]:
@torch.no_grad()
def bench_once_img2col_int8_vs_unfold(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
    iters=100,
    device="cuda",
):

    x_q = torch.randint(-128, 127, (N, Cin, H, W),
                        device=device, dtype=torch.int8)

    x_f16 = x_q.float().half()

    Ho = (H + 2 * Ph - Dh * (Kh - 1) - 1) // Sh + 1
    Wo = (W + 2 * Pw - Dw * (Kw - 1) - 1) // Sw + 1

    K = Cin * Kh * Kw
    M = N * Ho * Wo
    K_pad = ((K + 3) // 4) * 4  

    #  F.unfold 
    def _call_torch():
        return F.unfold(
            x_f16,
            kernel_size=(Kh, Kw),
            dilation=(Dh, Dw),
            padding=(Ph, Pw),
            stride=(Sh, Sw),
        )

    for _ in range(5):
        _ = _call_torch()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        unf = _call_torch()
    torch.cuda.synchronize()
    t_torch = (time.perf_counter() - t0) / iters

    N2, KC, L = unf.shape
    assert N2 == N
    assert KC == K
    assert L == Ho * Wo

    cols_ref = unf.permute(0, 2, 1).contiguous().view(M, K)  # [M, K]

    # img2col_int8
    if K_pad != K:
        cols_ref_pad = torch.zeros((M, K_pad), device=device, dtype=cols_ref.dtype)
        cols_ref_pad[:, :K] = cols_ref
    else:
        cols_ref_pad = cols_ref

    cols_ref_q = cols_ref_pad.to(torch.int8)
    
    def _call_triton():
        cols_q, (Ho2, Wo2) = img2col_int8(
            x_q,
            Kh, Kw,
            Sh, Sw,
            Ph, Pw,
            Dh, Dw,
            K_pad,
            BLOCK_M=BLOCK_M,
            BLOCK_K=BLOCK_K,
            num_warps=num_warps,
            num_stages=num_stages,
        )
        assert Ho2 == Ho and Wo2 == Wo
        assert cols_q.shape == (M, K_pad)
        return cols_q

    for _ in range(5):
        cols_q = _call_triton()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        cols_q = _call_triton()
    torch.cuda.synchronize()
    t_triton = (time.perf_counter() - t0) / iters

    # Error 
    cols_q_f32   = cols_q.float()
    cols_ref_f32 = cols_ref_q.float()

    diff = (cols_q_f32 - cols_ref_f32).abs()
    max_abs_err16 = diff.max().item()
    mean_abs_err16 = diff.mean().item()
    
    # bandwidth 
    bytes_moved = 2.0 * M * K_pad  # load + store int8
    bw_triton = bytes_moved / t_triton / 1e9
    bw_torch  = bytes_moved / t_torch  / 1e9

    return {
        "M": M, "K": K_pad, "N": N,
        "BLOCK_M": BLOCK_M,
        "BLOCK_N": 0,              
        "BLOCK_K": BLOCK_K,
        "num_warps": num_warps,
        "num_stages": num_stages,
        "t_triton_ms": t_triton * 1e3,
        "t_torch16_ms": t_torch * 1e3,
        "speed_vs_torch16": t_torch / t_triton,
        "bw_triton_GBs": bw_triton,
        "bw_torch16_GBs": bw_torch,
        "max_abs_err": max_abs_err16,
        "mean_abs_err": mean_abs_err16,
    }


In [15]:
@torch.no_grad()
def tune_img2col_int8_tiles_for_shape(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(1, 2, 4, 8),
    stages=(2, 3, 4),
    iters=200,
    device="cuda",
):
    records = []
    for BM in blocks_M:
        for BK in blocks_K:
            for W in warps:
                for S in stages:
                    try:
                        rec = bench_once_img2col_int8_vs_unfold(
                            N, Cin, H, W,
                            Kh, Kw,
                            Sh, Sw,
                            Ph, Pw,
                            Dh, Dw,
                            BLOCK_M=BM,
                            BLOCK_K=BK,
                            num_warps=W,
                            num_stages=S,
                            iters=iters,
                            device=device,
                        )
                    except RuntimeError as e:
                        print(f"[SKIP] BM={BM}, BK={BK}, W={W}, S={S}: {e}")
                        continue

                    print(
                        f"BM={BM}, BK={BK}, W={W}, S={S}: "
                        f"t_triton={rec['t_triton_ms']:.3f} ms, "
                        f"speed_vs_torch={rec['speed_vs_torch16']:.3f}x, "
                    )
                    records.append(rec)

    if not records:
        raise RuntimeError("No valid tile configs found for this IMG2COL shape")

    df = pd.DataFrame(records)
    return df


In [16]:
df_i2c = tune_img2col_int8_tiles_for_shape(
    N=16, Cin=1, H=256, W=256,
    Kh=11, Kw=11,
    Sh=1, Sw=1,
    Ph=5, Pw=5,
    Dh=1, Dw=1,
    blocks_M=(32, 64, 128, ),
    blocks_K=(32, 64, 128, ),
    warps=(2, 4, 8, 16),
    stages=(2, 3),
    iters=200,
    device="cuda",
)

BM=32, BK=32, W=2, S=2: t_triton=0.056 ms, speed_vs_torch=4.112x, 
BM=32, BK=32, W=2, S=3: t_triton=0.049 ms, speed_vs_torch=4.666x, 
BM=32, BK=32, W=4, S=2: t_triton=0.111 ms, speed_vs_torch=3.264x, 
BM=32, BK=32, W=4, S=3: t_triton=0.103 ms, speed_vs_torch=4.602x, 
BM=32, BK=32, W=8, S=2: t_triton=0.187 ms, speed_vs_torch=2.302x, 
BM=32, BK=32, W=8, S=3: t_triton=0.204 ms, speed_vs_torch=1.824x, 
BM=32, BK=32, W=16, S=2: t_triton=0.408 ms, speed_vs_torch=0.973x, 
BM=32, BK=32, W=16, S=3: t_triton=0.420 ms, speed_vs_torch=0.896x, 
BM=32, BK=64, W=2, S=2: t_triton=0.052 ms, speed_vs_torch=4.136x, 
BM=32, BK=64, W=2, S=3: t_triton=0.054 ms, speed_vs_torch=3.685x, 
BM=32, BK=64, W=4, S=2: t_triton=0.096 ms, speed_vs_torch=3.764x, 
BM=32, BK=64, W=4, S=3: t_triton=0.098 ms, speed_vs_torch=3.868x, 
BM=32, BK=64, W=8, S=2: t_triton=0.186 ms, speed_vs_torch=1.989x, 
BM=32, BK=64, W=8, S=3: t_triton=0.235 ms, speed_vs_torch=1.745x, 
BM=32, BK=64, W=16, S=2: t_triton=0.392 ms, speed_vs_torch=0

In [17]:
cols = [
    "BLOCK_M", "BLOCK_K",
    "num_warps", "num_stages",
    "t_triton_ms", "t_torch16_ms",
    "speed_vs_torch16",
     "mean_abs_err"
]

df_i2c_filtered = df_i2c[cols].sort_values("speed_vs_torch16", ascending=False).head(5).reset_index(drop=True)
df_i2c_filtered

,BLOCK_M,BLOCK_K,num_warps,num_stages,t_triton_ms,t_torch16_ms,speed_vs_torch16,mean_abs_err
0,32,32,2,3,0.048808,0.227726,4.665713,0.0
1,32,32,4,3,0.102678,0.472531,4.602085,0.0
2,64,32,4,3,0.095752,0.404809,4.227683,0.0
3,128,32,4,3,0.104053,0.437366,4.203297,0.0
4,32,64,2,2,0.052409,0.216752,4.135802,0.0


In [74]:
INT8_IMG2COL_BEST_BLOCK_M = 32
INT8_IMG2COL_BEST_BLOCK_N = 0
INT8_IMG2COL_BEST_BLOCK_K = 64
INT8_IMG2COL_BEST_WARPS   = 2
INT8_IMG2COL_BEST_STAGES  = 2